In [1]:
# set PATH
import sys
sys.path.insert(0, '../src')

In [2]:
from query_executor_get_raster import GetRasterExecutor

variable = "2m_temperature"
start_datetime = "2020-01-01 00:00:00"
end_datetime = "2023-12-31 23:00:00"
temporal_resolution = "hour"
temporal_aggregation = None
# Greenland
max_lat = 85
min_lat = 60
min_lon = -70
max_lon = -10

qe = GetRasterExecutor(
    variable=variable,
    start_datetime=start_datetime,
    end_datetime=end_datetime,
    temporal_resolution=temporal_resolution,
    temporal_aggregation=temporal_aggregation,
    min_lat=min_lat,
    max_lat=max_lat,
    min_lon=min_lon,
    max_lon=max_lon
)


qe.execute()

local files: ['/data/era5/raw/2m_temperature/2m_temperature-2020.nc', '/data/era5/raw/2m_temperature/2m_temperature-2021.nc', '/data/era5/raw/2m_temperature/2m_temperature-2022.nc', '/data/era5/raw/2m_temperature/2m_temperature-2023.nc']
api: []


<xarray.Dataset> Size: 7GB
Dimensions:    (time: 35064, latitude: 101, longitude: 241)
Coordinates:
  * longitude  (longitude) float32 964B -70.0 -69.75 -69.5 ... -10.25 -10.0
  * latitude   (latitude) float32 404B 85.0 84.75 84.5 84.25 ... 60.5 60.25 60.0
  * time       (time) datetime64[ns] 281kB 2020-01-01 ... 2023-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float64 7GB dask.array<chunksize=(8784, 101, 241), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-03 17:11:52 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [3]:
from metadata import Metadata

metadata = Metadata("/home/huan1531/iharp-query-executor/src/metadata.csv")


In [4]:
import pandas as pd

if temporal_aggregation is None:
    temporal_aggregation = "none"
spatial_aggregation=None
spatial_resolution=0.25
if spatial_aggregation is None:
    spatial_aggregation = "none"

df_overlap = metadata.df_meta[
    (metadata.df_meta["variable"] == variable)
    & (metadata.df_meta["min_lat"] <= max_lat)
    & (metadata.df_meta["max_lat"] >= min_lat)
    & (metadata.df_meta["min_lon"] <= max_lon)
    & (metadata.df_meta["max_lon"] >= min_lon)
    & (pd.to_datetime(metadata.df_meta["start_datetime"]) <= pd.to_datetime(end_datetime))
    & (pd.to_datetime(metadata.df_meta["end_datetime"]) >= pd.to_datetime(start_datetime))
    & (metadata.df_meta["temporal_resolution"] == temporal_resolution)
    & (metadata.df_meta["spatial_resolution"] == spatial_resolution)
    & (metadata.df_meta["temporal_aggregation"] == temporal_aggregation)
    & (metadata.df_meta["spatial_aggregation"] == spatial_aggregation)
]
df_overlap

,variable,start_datetime,end_datetime,max_lat,min_lat,max_lon,min_lon,temporal_resolution,temporal_aggregation,spatial_resolution,spatial_aggregation,file_path
36,2m_temperature,2020-01-01 00:00,2020-12-31 23:00,90,-90,180,-180,hour,none,0.25,none,/data/era5/raw/2m_temperature/2m_temperature-2...
37,2m_temperature,2021-01-01 00:00,2021-12-31 23:00,90,-90,180,-180,hour,none,0.25,none,/data/era5/raw/2m_temperature/2m_temperature-2...
38,2m_temperature,2022-01-01 00:00,2022-12-31 23:00,90,-90,180,-180,hour,none,0.25,none,/data/era5/raw/2m_temperature/2m_temperature-2...
39,2m_temperature,2023-01-01 00:00,2023-12-31 23:00,90,-90,180,-180,hour,none,0.25,none,/data/era5/raw/2m_temperature/2m_temperature-2...


In [5]:
import xarray as xr
from utils.const import lat_range, lon_range, time_resolution_to_freq
def gen_empty_xarray(
    min_lat,
    max_lat,
    min_lon,
    max_lon,
    start_datetime,
    end_datetime,
    temporal_resolution,
):
    lat_start = lat_range.searchsorted(min_lat, side="left")
    lat_end = lat_range.searchsorted(max_lat, side="right")
    lon_start = lon_range.searchsorted(min_lon, side="left")
    lon_end = lon_range.searchsorted(max_lon, side="right")
    ds_empty = xr.Dataset()
    ds_empty["time"] = pd.date_range(
        start=start_datetime,
        end=end_datetime,
        freq=time_resolution_to_freq(temporal_resolution),
    )
    ds_empty["latitude"] = lat_range[lat_start:lat_end]
    ds_empty["longitude"] = lon_range[lon_start:lon_end]
    return ds_empty

ds_query = gen_empty_xarray(
            min_lat, max_lat, min_lon, max_lon, start_datetime, end_datetime, temporal_resolution
        )
ds_query

<xarray.Dataset> Size: 283kB
Dimensions:    (time: 35064, latitude: 101, longitude: 241)
Coordinates:
  * time       (time) datetime64[ns] 281kB 2020-01-01 ... 2023-12-31T23:00:00
  * latitude   (latitude) float64 808B 60.0 60.25 60.5 60.75 ... 84.5 84.75 85.0
  * longitude  (longitude) float64 2kB -70.0 -69.75 -69.5 ... -10.5 -10.25 -10.0
Data variables:
    *empty*

In [6]:
import numpy as np
false_mask = xr.DataArray(
            np.zeros(
                (
                    ds_query.sizes["time"],
                    ds_query.sizes["latitude"],
                    ds_query.sizes["longitude"],
                ),
                dtype=bool,
            ),
            dims=["time", "latitude", "longitude"],
        )
false_mask

<xarray.DataArray (time: 35064, latitude: 101, longitude: 241)> Size: 853MB
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
...
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])
Dimensions without coordinates: time, latitude, longitude

In [7]:
row = df_overlap.iloc[0]
ds_meta = metadata._gen_xarray_for_meta_row(row)
mask = metadata._mask_query_with_meta(ds_query, ds_meta)
false_mask = false_mask | mask

In [8]:
ds_meta

<xarray.Dataset> Size: 88kB
Dimensions:    (time: 8784, latitude: 721, longitude: 1441)
Coordinates:
  * time       (time) datetime64[ns] 70kB 2020-01-01 ... 2020-12-31T23:00:00
  * latitude   (latitude) float64 6kB -90.0 -89.75 -89.5 ... 89.5 89.75 90.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.8 180.0
Data variables:
    *empty*

In [9]:
false_mask.where(false_mask == False, drop=True)

<xarray.DataArray (time: 26280, latitude: 101, longitude: 241)> Size: 5GB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * time       (time) datetime64[ns] 210kB 2021-01-01 ... 2023-12-31T23:00:00
  * latitude   (latitude) float64 808B 60.0 60.25 60.5 60.75 ... 84.5 84.75 85.0
  * longitude  (longitude) float64 2kB -70.0 -69.75 -69.5 ... -10.5 -10.25 -10.0